In [6]:
import pandas as pd
import json
from qto_buccaneer.utils._config_loader import load_config
from pathlib import Path
import os
from qto_buccaneer._utils.calculate.exp_data.metadata_filter_exp import MetadataFilter

In [7]:
file_path = "exp_data/ifc_model_metadata.json"

# First read the JSON file as a Python dictionary
with open(file_path, 'r') as f:
    data = json.load(f)

# Access the elements dictionary and convert to DataFrame
elements_df = pd.DataFrame.from_dict(data['elements'], orient='index')
elements_df.head()



,id,parent_id,GlobalId,IfcEntity,Classifications,Systems,Name,ConnectedFrom,ConnectedTo,ContainedInStructure,...,Pset_Enrichment.EBF,ePset_abstractBIM.ClassificationSIA416,ePset_abstractBIM.OriginalClassificationSIA,ePset_abstractBIM.OriginalGUIDS,Qto_SpaceBaseQuantities.Height,Qto_SpaceBaseQuantities.Perimeter,Qto_SpaceBaseQuantities.NetFloorArea,Qto_SpaceBaseQuantities.NetVolume,Pset_Enrichment.m2,Pset_Enrichment.RFB Nr.
1,1,NaN,21MDAeADb9799Web$9RSGv,IfcProject,[],[],Mustermodell V1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,166.0,02elX7_Bn8iOvJoCquoGkE,IfcCovering,[],[],CLADDING,[],[],[2h36Exz$16SQJ6pcny43V3],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,166.0,0yYm8GUG91FwVoYIMqgbFo,IfcCovering,[],[],CLADDING,[],[],[2h36Exz$16SQJ6pcny43V3],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,166.0,3nEtufSFj7bwK99Y7Xt8xx,IfcCovering,[],[],CLADDING,[],[],[2h36Exz$16SQJ6pcny43V3],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,164.0,0tkgbUMcT25OBOhCgbQt78,IfcCovering,[],[],FLOORING,[],[],[3_tSJLGEDBgPkOwRkkHz8p],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
config_path = Path("exp_data") / "00_workflow_config.yaml"
config = load_config(config_path)

metrics_config = config.get("metrics")
gf_gesamt_config = metrics_config.get("gf_gesamt")
print(gf_gesamt_config)

{'name': 'GF Gesamt', 'description': 'Die Gesamte Geschossfläche, durch die abstraction sind die Aussenwände mit einer Dicke von 35cm berechnet.', 'config': {'quantity_type': 'area', 'unit': 'm2', 'formula': 'GF - LUF', 'components': {'GF': {'filter': 'IfcEntity=IfcSpace AND Name=GrossArea', 'base_quantity': 'Qto_SpaceBaseQuantities.NetFloorArea'}, 'LUF': {'filter': 'IfcEntity=IfcSpace AND LongName=LUF', 'base_quantity': 'Qto_SpaceBaseQuantities.NetFloorArea'}}}}


In [9]:
def calculate_metric(df: pd.DataFrame, metric_config: dict, building: str) -> pd.DataFrame:
    """
    Calculate a metric based on the provided configuration.
    
    Args:
        df: DataFrame containing the IFC model data
        metric_config: Dictionary containing metric configuration
        building: Name of the building
        
    Returns:
        DataFrame with metric results
    """
    import time
    start_time = time.time()
    
    # Check if this is a formula-based calculation
    if 'formula' in metric_config['config']:
        # Calculate each component
        component_values = {}
        for component_name, component_config in metric_config['config']['components'].items():
            # Filter the DataFrame for this component
            filtered_df = MetadataFilter.filter_df_from_str(df, component_config['filter'])
            # Calculate the sum of the specified quantity
            component_values[component_name] = filtered_df[component_config['base_quantity']].sum()
        
        # Evaluate the formula
        formula = metric_config['config']['formula']
        # Replace component names with their values
        for component_name, value in component_values.items():
            formula = formula.replace(component_name, str(value))
        # Evaluate the formula safely
        try:
            value = eval(formula)
        except Exception as e:
            raise ValueError(f"Error evaluating formula '{formula}': {str(e)}")
    else:
        # Original simple calculation
        filtered_df = MetadataFilter.filter_df_from_str(df, metric_config['config']['filter'])
        value = filtered_df[metric_config['config']['base_quantity']].sum()
    
    # Create result DataFrame
    result = pd.DataFrame({
        'metric_name': [metric_config['name']],
        'value': [value],
        'unit': [metric_config['config']['unit']],
        'success': [True],
        'calculation_time': [time.time() - start_time],
        'building': [building],
        'description': [metric_config['description']],
        'formula': [metric_config['config']['formula']],
        'components': str([metric_config['config']['components']])

    })  
    
    return result

# Example usage:
metric_result = calculate_metric(elements_df, gf_gesamt_config, "Example Building")
metric_result



,metric_name,value,unit,success,calculation_time,building,description,formula,components
0,GF Gesamt,194.41235,m2,True,0.00453,Example Building,"Die Gesamte Geschossfläche, durch die abstract...",GF - LUF,[{'GF': {'filter': 'IfcEntity=IfcSpace AND Nam...


In [10]:
# Initialize an empty list to store all metric results
all_results = []

# Calculate all metrics and accumulate results
for metric_name, metric_config in metrics_config.items():
    metric_result = calculate_metric(elements_df, metric_config, "Example Building")
    all_results.append(metric_result)

# Combine all results into a single DataFrame
combined_results = pd.concat(all_results, ignore_index=True)
combined_results

,metric_name,value,unit,success,calculation_time,building,description,formula,components
0,HNF Total,55.218750,m2,True,0.002595,Example Building,Die Fläche aller Räume die nach SIA416 als HNF...,HNF,[{'HNF': {'filter': 'IfcEntity=IfcSpace AND Pr...
1,NNF Total,40.000000,m2,True,0.001868,Example Building,Die Fläche aller Räume die nach SIA416 als NNF...,NNF,[{'NNF': {'filter': 'IfcEntity=IfcSpace AND Pr...
2,VF Verkehrsfläche Neubau,11.625000,m2,True,0.002230,Example Building,Die Fläche aller Räume die nach SIA416 als VF ...,VF,[{'VF': {'filter': 'IfcEntity=IfcSpace AND Pre...
3,FF Funktionsfläche Neubau: m2,0.000000,m2,True,0.001680,Example Building,Die Fläche aller Räume die nach SIA416 als FF ...,FF,[{'FF': {'filter': 'IfcEntity=IfcSpace AND Pre...
4,GF Gesamt,194.412350,m2,True,0.002747,Example Building,"Die Gesamte Geschossfläche, durch die abstract...",GF - LUF,[{'GF': {'filter': 'IfcEntity=IfcSpace AND Nam...
5,% Anteil KF an GF,18.597769,%,True,0.002711,Example Building,Der Anteil der KF an der GF,(GF-net_floor_area) / GF * 100,[{'GF': {'filter': 'IfcEntity=IfcSpace AND Nam...
6,KF Konstruktionsfläche Total,38.318350,m2,True,0.002553,Example Building,Die Fläche aller Räume die nach SIA416 als KF ...,GF-net_floor_area,[{'GF': {'filter': 'IfcEntity=IfcSpace AND Nam...
7,GF oi Neubau,128.291100,m2,True,0.003261,Example Building,Die Geschossfläche oberirdisch,GFoi-LUFoi,[{'GFoi': {'filter': 'IfcEntity=IfcSpace AND N...
8,GF ui Neubau,66.121250,m2,True,0.003098,Example Building,Die Geschossfläche unterirdisch,GFui-LUFui,[{'GFui': {'filter': 'IfcEntity=IfcSpace AND N...
9,GV Neubau oi,452.335680,m3,True,0.001890,Example Building,Das Geschossvolumen oberirdisch,GVoi,[{'GVoi': {'filter': 'IfcEntity=IfcSpace AND N...


In [11]:
combined_results.to_excel("berechneten_metriken.xlsx")